#### load the dataset


In [1]:
### path = data/dataset
from PIL import Image
import os

In [14]:
from datasets import load_dataset

ds = load_dataset("sivan22/hebrew-handwritten-characters")

c:\Users\user\Desktop\נתניה\HebrewHandwrittenRecognition\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 5093/5093 [00:19<00:00, 255.08 examples/s]


In [39]:
# this cell is for testing

#open an image file from the dataset
img = Image.open("../data/dataset/TRAIN/0/0_1.png")

#display the image
# img.show()

#show image pixels
print(img.size)

#resize image to 40 by 40 pixels
img = img.resize((40, 40))
print(img.size)
img.show()


#convert image to rgba and print each pixel
img = img.convert('RGBA')

#get the pixel values of the image
pixels = list(img.getdata())
binary_array = []
for pixel in pixels:
    if pixel[0] ==  255 and pixel[1] == 255 and pixel[2] == 225:
        binary_array.append(0)
    else:
        binary_array.append(1)

#print all the non 0 pixels
print(binary_array)

#convert the binary array to an image
img = Image.new('1', (40, 40))
img.putdata(binary_array)
img.show()




(120, 125)
(40, 40)
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [43]:
# this cell is for testing

#open an image file from the dataset
img = Image.open("../data/dataset/TRAIN/0/0_1.png")

#display the image
# img.show()

#show image pixels
print(img.size)
img.show()
#resize image to 40 by 40 pixels
img = img.resize((40, 40))
print(img.size)
img.show()


#convert image to rgba and print each pixel
img = img.convert('RGBA')

#get the pixel values of the image
pixels = list(img.getdata())
binary_array = []
for pixel in pixels:
    if pixel[0] >= 128 and pixel[1] >= 128 and pixel[2] >= 128:
        binary_array.append(1)
    else:
        binary_array.append(0)

#print all the non 0 pixels
print(binary_array)

#convert the binary array to an image
img = Image.new('1', (40, 40))
img.putdata(binary_array)
img.show()




(120, 125)
(40, 40)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

### method to convert img to arr

In [28]:
def convert_img_to_binary_array(img):
    #convert image to rgba
    img = img.resize((40, 40))  
    img = img.convert('RGBA')

    #get the pixel values of the image
    pixels = list(img.getdata())
    binary_array = []
    for pixel in pixels:
        if pixel[0] >= 128 and pixel[1] >= 128 and pixel[2] >= 128:
            binary_array.append(0)
        else:
            binary_array.append(1)
    return binary_array

### create training and testing arrays

In [29]:
X_train = []
y_train = []

path = "../data/dataset/TRAIN/"
for folder in os.listdir(path):
    folder_path = os.path.join(path, folder)
    if not os.path.isdir(folder_path):
        continue
    if folder == "27":
        continue
    print(f"Processing folder: {folder}")
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        if os.path.isfile(file_path):
            try:
                img = Image.open(file_path)
                binary_array = convert_img_to_binary_array(img)
                X_train.append(binary_array)
                y_train.append(int(folder))
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")

Processing folder: 0
Processing folder: 1
Processing folder: 10
Processing folder: 11
Processing folder: 12
Processing folder: 13
Processing folder: 14
Processing folder: 15
Processing folder: 16
Processing folder: 17
Processing folder: 18
Processing folder: 19
Processing folder: 2
Processing folder: 20
Processing folder: 21
Processing folder: 22
Processing folder: 23
Processing folder: 24
Processing folder: 25
Processing folder: 26
Processing folder: 3
Processing folder: 4
Processing folder: 5
Processing folder: 6
Processing folder: 7
Processing folder: 8
Processing folder: 9


In [30]:
X_test = []
y_test = []
path = "../data/dataset/TEST/"
for folder in os.listdir(path):
    folder_path = os.path.join(path, folder)
    if not os.path.isdir(folder_path):
        continue
    if folder == "27":
        continue
    print(f"Processing folder: {folder}")
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        if os.path.isfile(file_path):
            try:
                img = Image.open(file_path)
                binary_array = convert_img_to_binary_array(img)
                X_test.append(binary_array)
                y_test.append(int(folder))
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")

Processing folder: 0
Processing folder: 1
Processing folder: 10
Processing folder: 11
Processing folder: 12
Processing folder: 13
Processing folder: 14
Processing folder: 15
Processing folder: 16
Processing folder: 17
Processing folder: 18
Processing folder: 19
Processing folder: 2
Processing folder: 20
Processing folder: 21
Processing folder: 22
Processing folder: 23
Processing folder: 24
Processing folder: 25
Processing folder: 26
Processing folder: 3
Processing folder: 4
Processing folder: 5
Processing folder: 6
Processing folder: 7
Processing folder: 8
Processing folder: 9


In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## split data randomly

In [32]:
# # connect train and test data
X = X_train + X_test
y = y_train + y_test


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Train size: {len(X_train)}")
print(f"Test size: {len(X_test)}")


Train size: 4058
Test size: 1015


## Create a SVM classifier

In [33]:
svm_model = svm.SVC()
svm_model.fit(X_train, y_train)
svm_predictions = svm_model.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_predictions)

## Create a KNN classifier

In [34]:
knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)
knn_predictions = knn_model.predict(X_test)
knn_accuracy = accuracy_score(y_test, knn_predictions)

## Create a Decision Tree classifier

In [35]:
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train, y_train)
dt_predictions = dt_model.predict(X_test)
dt_accuracy = accuracy_score(y_test, dt_predictions)

## Create a Random Forest classifier

In [36]:
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_predictions)

## Create a Naive Bayes classifier

In [37]:
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)
nb_predictions = nb_model.predict(X_test)
nb_accuracy = accuracy_score(y_test, nb_predictions)

# Compare the accuracies of the models


In [38]:
print("SVM Accuracy:", svm_accuracy)
print("KNN Accuracy:", knn_accuracy)
print("Decision Tree Accuracy:", dt_accuracy)
print("Random Forest Accuracy:", rf_accuracy)
print("Naive Bayes Accuracy:", nb_accuracy)

SVM Accuracy: 0.7497536945812808
KNN Accuracy: 0.5665024630541872
Decision Tree Accuracy: 0.4019704433497537
Random Forest Accuracy: 0.6926108374384237
Naive Bayes Accuracy: 0.34876847290640395


In [40]:
svm_accuracy = accuracy_score(y_test, svm_predictions)
knn_accuracy = accuracy_score(y_test, knn_predictions)
dt_accuracy = accuracy_score(y_test, dt_predictions)
rf_accuracy = accuracy_score(y_test, rf_predictions)
nb_accuracy = accuracy_score(y_test, nb_predictions)

svm_precision = precision_score(y_test, svm_predictions, average='weighted')
knn_precision = precision_score(y_test, knn_predictions, average='weighted')
dt_precision = precision_score(y_test, dt_predictions, average='weighted')
rf_precision = precision_score(y_test, rf_predictions, average='weighted')
nb_precision = precision_score(y_test, nb_predictions, average='weighted')

svm_recall = recall_score(y_test, svm_predictions, average='weighted')
knn_recall = recall_score(y_test, knn_predictions, average='weighted')
dt_recall = recall_score(y_test, dt_predictions, average='weighted')
rf_recall = recall_score(y_test, rf_predictions, average='weighted')
nb_recall = recall_score(y_test, nb_predictions, average='weighted')

svm_f1 = f1_score(y_test, svm_predictions, average='weighted')
knn_f1 = f1_score(y_test, knn_predictions, average='weighted')
dt_f1 = f1_score(y_test, dt_predictions, average='weighted')
rf_f1 = f1_score(y_test, rf_predictions, average='weighted')
nb_f1 = f1_score(y_test, nb_predictions, average='weighted')

performance_metrics = pd.DataFrame({
    'Algorithm': ['SVM', 'KNN', 'Decision Tree', 'Random Forest', 'Naive Bayes'],
    'Accuracy': [svm_accuracy, knn_accuracy, dt_accuracy, rf_accuracy, nb_accuracy],
    'Precision': [svm_precision, knn_precision, dt_precision, rf_precision, nb_precision],
    'Recall': [svm_recall, knn_recall, dt_recall, rf_recall, nb_recall],
    'F1-Score': [svm_f1, knn_f1, dt_f1, rf_f1, nb_f1]
})

print(performance_metrics)

c:\Users\user\Desktop\נתניה\HebrewHandwrittenRecognition\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


       Algorithm  Accuracy  Precision    Recall  F1-Score
0            SVM  0.749754   0.760278  0.749754  0.748954
1            KNN  0.566502   0.613491  0.566502  0.556145
2  Decision Tree  0.401970   0.406591  0.401970  0.401478
3  Random Forest  0.692611   0.699681  0.692611  0.687746
4    Naive Bayes  0.348768   0.388866  0.348768  0.310005
